# <center>Watson</center>

In [ ]:
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, SyntaxOptions, SyntaxOptionsTokens, SentimentOptions
import pandas as pd
import os
from warnings import simplefilter
import re
import Kappas as kp

simplefilter(action='ignore', category=FutureWarning)

## Predicción (API Watson IBM)

In [ ]:
claveAPI = '...'
url = '...'

def watson_auth(claveAPI,url):
    authenticator = IAMAuthenticator(claveAPI)
    service = NaturalLanguageUnderstandingV1(
         version='2020-10-09',
         authenticator=authenticator)
    service.set_service_url(url)
    return service


In [ ]:
service = watson_auth(claveAPI,url)

def sentiment_analysis(service,texto,lang,targets):
    texto = texto.replace('@realdonaldtrump', 'realdonaldtrump')
    texto = re.sub(r'\S*@\S*\s?', '', texto)
    texto = re.sub(r'\S*#\S*\s?', '', texto)
    texto = re.sub(r'\S*http\S*\s?', '', texto)
    return service.analyze(
                        text=texto,
                        language= lang,
                        features= {
                            "sentiment": {
                                "targets": targets,
                                "document": False
                            }       
                        }
                            ).get_result()

In [ ]:
def watson_analysis(data, target):
    results = []
    cont = 0
    for texto in data:
        if cont % 100 == 0:
            print(cont)
        cont+=1

        try:
            output = sentiment_analysis(service, texto, "en", [target])
            result = output['sentiment']['targets'][0]['label']
            #score = output['sentiment']['targets'][0]['score']
            results.append([result, texto])
        except Exception as e:
            print("Error processing: " + texto)
            #results.append([999, texto])
            print(cont)
            print(e)
    return results

In [ ]:
target = 'realdonaldtrump'
path = "https://raw.githubusercontent.com/raqblanc/analsis-sentimiento-contexto-competitivo/main/data/trump-2020.csv"
df_real = pd.read_csv(path)
results = watson_analysis(df_real['texto'].to_list(), target)

In [ ]:
df_watson = pd.DataFrame(results, columns=['valor', 'texto'])
df_watson.loc[df_watson.valor == 'positive', 'valor'] = 1
df_watson.loc[df_watson.valor == 'negative', 'valor'] = -1
df_watson.loc[df_watson.valor == 'neutral', 'valor'] = 0

df = df_real.merge(df_watson,on='texto',how='right')

In [ ]:
y_real = df['real'].tolist()
y_pred = df['valor'].tolist()
kp.calcula_kappas(y_real,y_pred)